In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from collections import defaultdict
import traceback
import os
import json

In [2]:
BEFORE_2023 = True
SHOW_FEATURE_SETS = False

In [3]:
# path to datasets
ver = "v12"
# count_bucket_name = "pheno-master-data-collection-v1.2-stage-eu-west-1"
# count_bucket_name = "pheno-master-data-collection-v1-prod-eu-west-1"
# count_bucket_name = "pheno-academic-collection-v1-prod-eu-west-1"
# count_bucket_name = "pheno-academic-collection-v1-prod-us-east-1"
# count_bucket_name = "pheno-master-data-collection-v1.2-prod-eu-west-1"
count_bucket_name = "pheno-academic-collection-v1.2-prod-eu-west-1"
master_path = f"s3://{count_bucket_name}"
output_path = f's3://ds-users/mariag/data_counts/{count_bucket_name}_{ver}_before_2023_{BEFORE_2023}_show_feature_{SHOW_FEATURE_SETS}.csv'

In [4]:
# link to the google sheet
sheet_id = '1l8gf56ifdBwAptohozsdVidx9dUS-yNx2qxamYsooTw'
sheet_name = 'Sheet1'
url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}"
df_feature_set = pd.read_csv(url, dtype=str).fillna(pd.NA)

    
df_feature_set[['dataset_id', 'dataset']] = df_feature_set[['dataset_id', 'dataset']].ffill()
df_feature_set_ready = df_feature_set[pd.notna(df_feature_set.status)].query('status != "Not relevant v1.2"')
df_feature_set_ready.head()

,dataset_id,dataset,feature_set,Data owner,status,pheno-big-data-prod-us-east-1_relative_path,contains_sensitive,notes,count_parquet_relative_path,count_field_name,data points,participants,first_date,last_date,dict_check,demo
0,0,population,population,Maria,DS Owner Finished for v1.2,<NA>,<NA>,<NA>,/population/population.parquet,sex,11179,11179,2018-12-05,2023-03-19,<NA>,<NA>
1,1,events,events,Maria,DS Owner Finished for v1.2,<NA>,<NA>,<NA>,/events/events.parquet,research_stage_date,23300,11179,2018-12-05,2023-03-19,<NA>,<NA>
2,2,anthropometrics,anthropometrics,Adam,DS Owner Finished for v1.2,<NA>,<NA>,<NA>,/anthropometrics/anthropometrics.parquet,bmi,13568,10781,2018-11-22,2023-02-07,<NA>,<NA>
3,3,fundus,fundus,Hagai,DS Owner Finished for v1.2,<NA>,<NA>,<NA>,/fundus/fundus.parquet,"fundus_image_left, fundus_image_right",7079,7049,2021-02-17,2022-12-04,<NA>,<NA>
4,3,fundus,microvasculature,<NA>,DS Owner Finished for v1.2,<NA>,<NA>,<NA>,/fundus/microvasculature.parquet,"vessel_density_left, vessel_density_right",7079,7049,2021-02-17,2022-12-04,<NA>,<NA>


In [5]:
# check for duplicates and remove features without parquet path or count field
print(df_feature_set_ready.shape)
df = df_feature_set_ready.query('@pd.notna(count_parquet_relative_path) and @pd.notna(count_field_name)') # in @none
print(df.shape)
df_no_dup = df.drop_duplicates(['dataset', 'feature_set', 'count_parquet_relative_path', 'count_field_name'])
print(df_no_dup.shape)

(115, 16)
(109, 16)
(109, 16)


In [42]:
count_dict = defaultdict(dict)
for index, row in df_no_dup.iterrows():
    try:
        dataset = row['dataset']
        if dataset == 'curated_phenotypes':
            continue
        feature_set = row['feature_set']
        
        #read parquet file
        full_path = f"{master_path}{row['count_parquet_relative_path']}"
        df_parquet = pd.read_parquet(full_path).reset_index()
        
        if dataset=='medications':
            row['count_field_name'] = 'medication'
        res_dict =  {
            "df_parquet_shape": df_parquet.shape[0],
            "count_field_name": row['count_field_name'],
            "count_parquet_relative_path": row['count_parquet_relative_path'],
            "particioant_id_nunique": df_parquet['participant_id'].nunique()
            }
        
        max_col = ''
        max_count = 0
        
        for col in row['count_field_name'].split(','):
            # remove white space
            col = col.strip()
            if col not in df_parquet.columns:
                print (f"ERROR: {col} not in {full_path} : {df_parquet.columns}")
                
            no_na_df = df_parquet.dropna(subset=[col]).copy()
            
            if dataset != 'events': 
                date_col = 'collection_date'
            else: 
                date_col = 'research_stage_date'
                
            if date_col not in no_na_df.columns:
                date_col = 'collection_timestamp'
            if date_col not in no_na_df.columns:
                first_date = None
                last_date = None
            else:   
                if BEFORE_2023:
                    # filter date_col before 2023-01-01
                    no_na_df[date_col] = pd.to_datetime(no_na_df[date_col])
                    no_na_df = no_na_df[no_na_df[date_col] < '2023-01-01']
                first_date = no_na_df[date_col].min() # first date
                last_date = no_na_df[date_col].max() # last date

                    
            nunique_participants_not_na = no_na_df.participant_id.nunique() # number of unique participants
            count_col_notna = no_na_df.shape[0] # number of not na entries
            
            if no_na_df.shape[0] > max_count:
                max_count = no_na_df.shape[0]
                max_col = col
                
                
            res_dict[col] = {
                "count_col_notna": count_col_notna,  
                "nunique_participants_not_na" : nunique_participants_not_na,
                "first_date": first_date,
                "last_date": last_date,
            }
        del df_parquet
        res_dict['max_col'] = max_col
        count_dict[f"{int(row['dataset_id'])}-{dataset}"][feature_set] = res_dict
        del no_na_df
        
        
    except Exception as e: 
        print(row['dataset'], full_path)
        print(traceback.format_exc())

sleep s3://pheno-academic-collection-v1.2-prod-eu-west-1/sleep/raw/sleep.parquet
Traceback (most recent call last):
  File "/tmp/ipykernel_3603/1176001438.py", line 9, in <module>
    df_parquet = pd.read_parquet(full_path).reset_index()
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ec2-user/miniconda3/lib/python3.11/site-packages/pandas/io/parquet.py", line 670, in read_parquet
    return impl.read(
           ^^^^^^^^^^
  File "/home/ec2-user/miniconda3/lib/python3.11/site-packages/pandas/io/parquet.py", line 272, in read
    pa_table = self.api.parquet.read_table(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/ec2-user/miniconda3/lib/python3.11/site-packages/pyarrow/parquet/core.py", line 2956, in read_table
    dataset = _ParquetDatasetV2(
              ^^^^^^^^^^^^^^^^^^
  File "/home/ec2-user/miniconda3/lib/python3.11/site-packages/pyarrow/parquet/core.py", line 2507, in __init__
    self._dataset = ds.dataset(path_or_paths, filesystem=filesystem,
         

In [43]:

full_data = list()
for k, v in count_dict.items():
    for kk, vv in v.items():
        data = list()
        data.append(k)
        data.append(kk)
        col = vv['max_col']
        data.append(vv[col]["count_col_notna"])
        data.append(vv[col]["nunique_participants_not_na"])
        data.append(vv[col]["first_date"])
        data.append(vv[col]["last_date"])
        full_data.append(data)
        


In [44]:
full_data

[['0-population', 'population', 12014, 12014, None, None],
 ['1-events',
  'events',
  20723,
  10434,
  Timestamp('2018-12-05 00:00:00'),
  Timestamp('2022-12-29 00:00:00')],
 ['2-anthropometrics',
  'anthropometrics',
  12925,
  10387,
  Timestamp('2018-11-22 00:00:00'),
  Timestamp('2022-12-29 00:00:00')],
 ['3-fundus',
  'fundus',
  7068,
  7038,
  Timestamp('2021-02-17 00:00:00'),
  Timestamp('2022-12-04 00:00:00')],
 ['3-fundus', 'microvasculature', 7068, 7038, None, None],
 ['3-fundus',
  'images',
  7068,
  7038,
  Timestamp('2021-02-17 00:00:00'),
  Timestamp('2022-12-04 00:00:00')],
 ['3-fundus', 'segmented_images', 7068, 7038, None, None],
 ['4-liver_ultrasound',
  'liver_ultrasound',
  26684,
  8229,
  Timestamp('2020-02-20 00:00:00'),
  Timestamp('2022-12-29 00:00:00')],
 ['5-diet_logging',
  'diet_logging',
  149912,
  9531,
  Timestamp('2019-09-01 00:00:00'),
  Timestamp('2022-12-31 00:00:00')],
 ['5-diet_logging',
  'diet_logging_events',
  2640312,
  10197,
  Timestamp

In [45]:
df_res = pd.DataFrame(full_data)
df_res.columns = ['dataset', 'feature_set', 'count_col_notna', 'nunique_participants_not_na', 'first_date', 'last_date']
df_res.head()


,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
0,0-population,population,12014,12014,None,None
1,1-events,events,20723,10434,2018-12-05 00:00:00,2022-12-29 00:00:00
2,2-anthropometrics,anthropometrics,12925,10387,2018-11-22 00:00:00,2022-12-29 00:00:00
3,3-fundus,fundus,7068,7038,2021-02-17 00:00:00,2022-12-04 00:00:00
4,3-fundus,microvasculature,7068,7038,None,None


In [46]:
# df_res.to_csv('pheno_stage_data_collection_v1_2_full_count.csv', index=False)
# df_res.to_csv('pheno_stage_data_collection_v1_2_before_2023_count.csv', index=False)

In [47]:
df_res.query('dataset == "17-cgm"')

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
45,17-cgm,cgm,9497,9431,2019-01-07 00:14:00+02:00,2022-12-30 00:14:00+02:00
46,17-cgm,timeseries,9497,9431,2019-01-07 00:14:00+02:00,2022-12-30 00:14:00+02:00
47,17-cgm,iglu,9497,9431,None,None
48,17-cgm,iglu_daily,104845,9431,2019-01-07 00:00:00,2022-12-31 00:00:00


In [48]:
if BEFORE_2023:
    df_res.loc[df_res['dataset']== '0-population', 'count_col_notna'] =  df_res.loc[df_res['dataset']== '1-events', 'nunique_participants_not_na'].values
    df_res.loc[df_res['dataset']== '0-population', 'nunique_participants_not_na'] =  df_res.loc[df_res['dataset']== '1-events', 'nunique_participants_not_na'].values
    k = 'population'
    v = 'events'
    df_res.loc[df_res['feature_set']== k, 'first_date'] =  df_res.loc[df_res['feature_set']== v, 'first_date'].values
    df_res.loc[df_res['feature_set']== k, 'last_date'] = df_res.loc[df_res['feature_set']== v, 'last_date'].values

In [49]:

df_res.loc[df_res['dataset']== '100-curated_phenotypes', 'first_date'] =  df_res.loc[df_res['dataset']== '1-events', 'first_date'].values[0]
df_res.loc[df_res['dataset']== '100-curated_phenotypes', 'last_date'] = df_res.loc[df_res['dataset']== '1-events','last_date'].values[0]

if BEFORE_2023:
    df_res.loc[df_res['dataset'] == '100-curated_phenotypes', 'count_col_notna'] =  df_res.loc[df_res['dataset'] == '100-curated_phenotypes', 'count_col_notna'].clip(upper=df_res.loc[df_res['dataset'] == '1-events', 'count_col_notna'].iloc[0])
    df_res.loc[df_res['dataset'] == '100-curated_phenotypes', 'nunique_participants_not_na'] =  df_res.loc[df_res['dataset'] == '100-curated_phenotypes', 'nunique_participants_not_na'].clip(upper=df_res.loc[df_res['dataset'] == '1-events', 'nunique_participants_not_na'].iloc[0])
    

df_res.tail(2)

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
87,70-oral_microbiome,humann,8816,8816,2019-03-11 00:00:00,2022-12-21 00:00:00
88,70-oral_microbiome,humann_aggregated,8816,8816,2019-03-11 00:00:00,2022-12-21 00:00:00


In [50]:
if BEFORE_2023 and not SHOW_FEATURE_SETS:
    df_res = df_res[pd.notna(df_res['last_date'])]

In [51]:

dict_dataset = {
    'population': 'events',
    'microvasculature': 'fundus',
    'segmented_images': 'fundus',
    'hrv': 'sleep',
    'hrv_raw': 'sleep',
    'pca': 'human_genetics',
    'iglu': 'iglu_daily', 
    
}
for k, v in dict_dataset.items():
    df_res.loc[df_res['feature_set']== k, 'first_date'] =  df_res.loc[df_res['feature_set']== v, 'first_date'].values
    df_res.loc[df_res['feature_set']== k, 'last_date'] = df_res.loc[df_res['feature_set']== v, 'last_date'].values

df_res.head()


,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
0,0-population,population,12014,12014,2018-12-05 00:00:00,2022-12-29 00:00:00
1,1-events,events,20723,10434,2018-12-05 00:00:00,2022-12-29 00:00:00
2,2-anthropometrics,anthropometrics,12925,10387,2018-11-22 00:00:00,2022-12-29 00:00:00
3,3-fundus,fundus,7068,7038,2021-02-17 00:00:00,2022-12-04 00:00:00
4,3-fundus,microvasculature,7068,7038,2021-02-17 00:00:00,2022-12-04 00:00:00


In [52]:
df_res.tail(2)


,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
87,70-oral_microbiome,humann,8816,8816,2019-03-11 00:00:00,2022-12-21 00:00:00
88,70-oral_microbiome,humann_aggregated,8816,8816,2019-03-11 00:00:00,2022-12-21 00:00:00


In [53]:
df_res.query('dataset == "17-cgm"')

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
45,17-cgm,cgm,9497,9431,2019-01-07 00:14:00+02:00,2022-12-30 00:14:00+02:00
46,17-cgm,timeseries,9497,9431,2019-01-07 00:14:00+02:00,2022-12-30 00:14:00+02:00
47,17-cgm,iglu,9497,9431,2019-01-07 00:00:00,2022-12-31 00:00:00
48,17-cgm,iglu_daily,104845,9431,2019-01-07 00:00:00,2022-12-31 00:00:00


In [54]:
import datetime

# Create a function to convert the date string to a datetime object
def convert_date_string(date_string):
    if date_string is None or date_string == 'None' or date_string == 'NaT' or date_string == 'nan':
        return None
    else:
        date_string_without_timezone = date_string[:19]
        datetime_object = datetime.datetime.strptime(date_string_without_timezone, '%Y-%m-%d %H:%M:%S')
        date = datetime_object.date()
        return date
    
    


In [55]:
df_convert = df_res.copy()


df_convert["first_date"] = df_convert["first_date"].apply(str)
df_convert["last_date"] = df_convert["last_date"].apply(str)

# Convert all the dates in the DataFrame to datetime objects
df_convert["first_date"] = df_convert["first_date"].apply(convert_date_string)
df_convert["last_date"] = df_convert["last_date"].apply(convert_date_string)

In [56]:
df_convert.query('dataset == "17-cgm"')

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
45,17-cgm,cgm,9497,9431,2019-01-07,2022-12-30
46,17-cgm,timeseries,9497,9431,2019-01-07,2022-12-30
47,17-cgm,iglu,9497,9431,2019-01-07,2022-12-31
48,17-cgm,iglu_daily,104845,9431,2019-01-07,2022-12-31


In [57]:
df_convert.shape

(89, 6)

In [58]:
df_convert.head(50)

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
0,0-population,population,12014,12014,2018-12-05,2022-12-29
1,1-events,events,20723,10434,2018-12-05,2022-12-29
2,2-anthropometrics,anthropometrics,12925,10387,2018-11-22,2022-12-29
3,3-fundus,fundus,7068,7038,2021-02-17,2022-12-04
4,3-fundus,microvasculature,7068,7038,2021-02-17,2022-12-04
5,3-fundus,images,7068,7038,2021-02-17,2022-12-04
6,3-fundus,segmented_images,7068,7038,2021-02-17,2022-12-04
7,4-liver_ultrasound,liver_ultrasound,26684,8229,2020-02-20,2022-12-29
8,5-diet_logging,diet_logging,149912,9531,2019-09-01,2022-12-31
9,5-diet_logging,diet_logging_events,2640312,10197,2019-09-01,2023-03-21


In [59]:
df_convert.tail(50)

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date
39,15-ecg,ecg,9702,8085,2019-08-13,2022-12-29
40,15-ecg,raw_timeseries_wide,9441,7888,2019-08-13,2022-12-29
41,15-ecg,processed_timeseries_wide,9172,7711,2019-08-13,2022-12-29
42,15-ecg,raw_timeseries_long,9441,7888,2019-08-13,2022-12-29
43,15-ecg,processed_timeseries_long,9172,7711,2019-08-13,2022-12-29
44,16-blood_tests,blood_tests,29074,10252,2010-01-06,2022-12-29
45,17-cgm,cgm,9497,9431,2019-01-07,2022-12-30
46,17-cgm,timeseries,9497,9431,2019-01-07,2022-12-30
47,17-cgm,iglu,9497,9431,2019-01-07,2022-12-31
48,17-cgm,iglu_daily,104845,9431,2019-01-07,2022-12-31


In [60]:
# Function to split 'dataset' column
def split_dataset(value):
    try:
        id_, dataset = value.split('-', 1)
    except ValueError:
        id_ = None
        dataset = value
    return pd.Series([id_, dataset])


In [61]:
df = df_convert.copy()

In [62]:

# Apply the function
df[['id', 'dataset']] = df['dataset'].apply(split_dataset)


/tmp/ipykernel_3603/3306964056.py:2: FutureWarning: Returning a DataFrame from Series.apply when the supplied function returns a Series is deprecated and will be removed in a future version.
  df[['id', 'dataset']] = df['dataset'].apply(split_dataset)


In [63]:
df.head()

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date,id
0,population,population,12014,12014,2018-12-05,2022-12-29,0
1,events,events,20723,10434,2018-12-05,2022-12-29,1
2,anthropometrics,anthropometrics,12925,10387,2018-11-22,2022-12-29,2
3,fundus,fundus,7068,7038,2021-02-17,2022-12-04,3
4,fundus,microvasculature,7068,7038,2021-02-17,2022-12-04,3


In [64]:
df.tail()

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date,id
84,oral_microbiome,fastqc,8816,8816,2019-03-11,2022-12-21,70
85,oral_microbiome,metaphlan,8816,8816,2019-03-11,2022-12-21,70
86,oral_microbiome,metaphlan_aggregated,8816,8816,2019-03-11,2022-12-21,70
87,oral_microbiome,humann,8816,8816,2019-03-11,2022-12-21,70
88,oral_microbiome,humann_aggregated,8816,8816,2019-03-11,2022-12-21,70


In [65]:
# Function to format the numbers
def format_number(x):
    return f'{int(x):03d}'

# Apply the function to the desired column
df['id'] = df['id'].apply(format_number)
 
df.head()


,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date,id
0,population,population,12014,12014,2018-12-05,2022-12-29,000
1,events,events,20723,10434,2018-12-05,2022-12-29,001
2,anthropometrics,anthropometrics,12925,10387,2018-11-22,2022-12-29,002
3,fundus,fundus,7068,7038,2021-02-17,2022-12-04,003
4,fundus,microvasculature,7068,7038,2021-02-17,2022-12-04,003


In [66]:
df.head(50)

,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date,id
0,population,population,12014,12014,2018-12-05,2022-12-29,000
1,events,events,20723,10434,2018-12-05,2022-12-29,001
2,anthropometrics,anthropometrics,12925,10387,2018-11-22,2022-12-29,002
3,fundus,fundus,7068,7038,2021-02-17,2022-12-04,003
4,fundus,microvasculature,7068,7038,2021-02-17,2022-12-04,003
5,fundus,images,7068,7038,2021-02-17,2022-12-04,003
6,fundus,segmented_images,7068,7038,2021-02-17,2022-12-04,003
7,liver_ultrasound,liver_ultrasound,26684,8229,2020-02-20,2022-12-29,004
8,diet_logging,diet_logging,149912,9531,2019-09-01,2022-12-31,005
9,diet_logging,diet_logging_events,2640312,10197,2019-09-01,2023-03-21,005


In [67]:
df.tail(20)


,dataset,feature_set,count_col_notna,nunique_participants_not_na,first_date,last_date,id
69,psychological_and_social_health,psychological_and_social_health,14861,9262,2019-01-09,2022-12-31,057
70,health_and_medical_history,ukbb,14861,9262,2019-01-09,2022-12-31,058
71,health_and_medical_history,initial_medical,10197,10197,2018-11-21,2022-12-31,058
72,participant_relation,participant relation,3739,3106,2019-02-05,2022-12-29,045
73,rna_seq,rna_seq,3958,3863,2021-02-28,2022-06-21,024
74,rna_seq,raw_fastq,3958,3863,2021-02-28,2022-06-21,024
75,rna_seq,trimmed_fastq,3958,3863,2021-02-28,2022-06-21,024
76,rna_seq,aligned_bam,3958,3863,2021-02-28,2022-06-21,024
77,rna_seq,deduplicated_bam,3958,3863,2021-02-28,2022-06-21,024
78,rna_seq,multiqc,3958,3863,2021-02-28,2022-06-21,024


In [68]:
# df.to_csv('s3://ds-users/mariag/pheno_stage_data_collection_v1_2_count_converted_fulls.csv', index=False)
# df.to_csv('s3://ds-users/mariag/pheno_stage_data_collection_v1_2_count_converted_before_2023.csv', index=False)

In [69]:
# for each daatset group by dataset and get the min and max date and max count and max participant
df_group = df.groupby('dataset').agg({'first_date': 'min', 'last_date': 'max', 'count_col_notna': 'max', 'nunique_participants_not_na': 'max', 'id':'first'}).reset_index()

In [70]:
df_group['int_id'] = df_group['id'].astype(int)
df_group.sort_values('int_id', inplace=True)
df_group


,dataset,first_date,last_date,count_col_notna,nunique_participants_not_na,id,int_id
27,population,2018-12-05,2022-12-29,12014,12014,000,0
11,events,2018-12-05,2022-12-29,20723,10434,001,1
0,anthropometrics,2018-11-22,2022-12-29,12925,10387,002,2
13,fundus,2021-02-17,2022-12-04,7068,7038,003,3
19,liver_ultrasound,2020-02-20,2022-12-29,26684,8229,004,4
9,diet_logging,2019-09-01,2023-03-21,6483638,10696,005,5
15,hand_grip,2018-12-27,2022-12-29,12894,10371,006,6
1,blood_pressure,2018-12-27,2022-12-29,12946,10398,007,7
30,serum_lipidomics,2019-02-19,2021-08-08,6321,6144,008,8
32,sleep,2020-01-15,2022-12-27,21372,7062,009,9


In [71]:
# df_group1 = df_group.copy()
# df_group2 = df_group.copy()

In [75]:
# count_bucket_name = "pheno-master-data-collection-v1.2-prod-eu-west-1"
# output_path = f'{count_bucket_name}_{ver}_before_2023_{BEFORE_2023}_show_feature_{SHOW_FEATURE_SETS}.csv'
# df_group1.to_csv(output_path, index=False)

# count_bucket_name = "pheno-academic-collection-v1.2-prod-eu-west-1"
# output_path = f'{count_bucket_name}_{ver}_before_2023_{BEFORE_2023}_show_feature_{SHOW_FEATURE_SETS}.csv'
# df_group2.to_csv(output_path, index=False)

In [72]:
# df_group.to_csv('s3://ds-users/mariag/pheno_stage_data_collection_v1_2_count_converted_full_grouped.csv', index=False)
# df_group.to_csv('s3://ds-users/mariag/pheno_stage_data_collection_v1_2_count_converted_before2023_grouped.csv', index=False)
df_group.to_csv(output_path, index=False)

PermissionError: Access Denied

In [ ]:
if not SHOW_FEATURE_SETS: 
    df  = df_group.copy()
    # Reorder the columns
    df = df[['id', 'dataset',  'count_col_notna', 'nunique_participants_not_na', 'first_date', 'last_date']] #'feature_set',
else: 
    # Reorder the columns
    df = df[['id', 'dataset',  'feature_set', 'count_col_notna', 'nunique_participants_not_na', 'first_date', 'last_date']] 

# Combine 'count_col_notna' and 'nunique_participants_not_na' columns
df['data points (participants)'] = df['count_col_notna'].astype(str) + " (" + df['nunique_participants_not_na'].astype(str) + ")"

# Drop the original columns
df = df.drop(columns=['count_col_notna', 'nunique_participants_not_na'])
df.head()

,id,dataset,feature_set,first_date,last_date,data points (participants)
0,000,population,population,2018-12-05,2022-12-29,11179 (11179)
1,001,events,events,2018-12-05,2022-12-29,20773 (10466)
2,002,anthropometrics,anthropometrics,2018-11-22,2022-12-29,12963 (10419)
3,003,fundus,fundus,2021-02-17,2022-12-04,7079 (7049)
4,003,fundus,microvasculature,2021-02-17,2022-12-04,7079 (7049)


In [ ]:
df.tail()

,id,dataset,feature_set,first_date,last_date,data points (participants)
46,017,cgm,timeseries,2019-01-07,2022-12-30,9524 (9458)
47,017,cgm,iglu,2019-01-07,2022-12-31,9524 (9458)
48,017,cgm,iglu_daily,2019-01-07,2022-12-31,105149 (9458)
49,022,bone_density,bone_density,2020-06-09,2022-12-29,9052 (8477)
50,018,medications,medications,2018-10-18,2022-12-31,14308 (10159)


In [ ]:

# Reorder the columns
# df = df[['id', 'dataset', 'data points (participants)', 'first_date', 'last_date']]#'feature_set',

# Remove '_' from column names
df.columns = df.columns.str.replace('_', ' ')

# Add 'tabular data' column with "✓" for all rows
df['tabular data'] = '✓'

# Add 'time series data' column with "✓" for specified rows
df['time series data'] = ''
df.loc[df['dataset'] == 'diet logging', 'time series data'] = '✓'
df.loc[(df['dataset'].isin(['sleep', 'ecg', 'cgm'])), 'time series data'] = '✓'

# Add 'image data' column with "✓" for specified rows
df['image data'] = ''
df.loc[(df['dataset'] == 'fundus') , 'image data'] = '✓'



In [ ]:
# Convert each value in the 'dataset' column to a link
df['dataset'] = df.apply(lambda row: f'[{row["dataset"]}](datasets/{row["id"]}-{row["dataset"]}.html)', axis=1)

# Convert the DataFrame to a markdown table
markdown_table = df.to_markdown(index=False)

In [ ]:
stop!

SyntaxError: invalid syntax (3319058519.py, line 1)

In [ ]:

# Save the markdown table to a .md file
with open('datasets.md', 'w') as f:
    f.write(markdown_table)
    f.write('\n\n')
    f.write(': Available Datasets {.striped .hover tbl-colwidths="[5, 37, 20 ,20, 20, 5, 5, 5]"}')
    
# Check the first few lines of the markdown table
print('\n'.join(markdown_table.split('\n')[:10]))


|   id | dataset                                                                              | first date   | last date   | data points (participants)   | tabular data   | time series data   | image data   |
|-----:|:-------------------------------------------------------------------------------------|:-------------|:------------|:-----------------------------|:---------------|:-------------------|:-------------|
|  000 | [population](datasets/000-population.html)                                           | 2018-12-05   | 2022-12-29  | 10434 (10434)                | ✓              |                    |              |
|  001 | [events](datasets/001-events.html)                                                   | 2018-12-05   | 2022-12-29  | 20723 (10434)                | ✓              |                    |              |
|  002 | [anthropometrics](datasets/002-anthropometrics.html)                                 | 2018-11-22   | 2022-12-29  | 12925 (10387)                | ✓       

In [ ]:
print('\n'.join(markdown_table.split('\n')[:20]))

|   id | dataset                                                                              | first date   | last date   | data points (participants)   | tabular data   | time series data   | image data   |
|-----:|:-------------------------------------------------------------------------------------|:-------------|:------------|:-----------------------------|:---------------|:-------------------|:-------------|
|  000 | [population](datasets/000-population.html)                                           | 2018-12-05   | 2022-12-29  | 10434 (10434)                | ✓              |                    |              |
|  001 | [events](datasets/001-events.html)                                                   | 2018-12-05   | 2022-12-29  | 20723 (10434)                | ✓              |                    |              |
|  002 | [anthropometrics](datasets/002-anthropometrics.html)                                 | 2018-11-22   | 2022-12-29  | 12925 (10387)                | ✓       

In [ ]:
print('\n'.join(markdown_table.split('\n')[-20:]))

|  016 | [blood_tests](datasets/016-blood_tests.html)                                         | 2010-01-06   | 2022-12-29  | 29074 (10252)                | ✓              |                    |              |
|  017 | [cgm](datasets/017-cgm.html)                                                         | 2019-01-07   | 2022-12-31  | 104845 (9431)                | ✓              | ✓                  |              |
|  018 | [medications](datasets/018-medications.html)                                         | 2018-10-18   | 2022-12-31  | 43417 (10787)                | ✓              |                    |              |
|  021 | [medical_conditions](datasets/021-medical_conditions.html)                           | 2018-11-21   | 2022-12-31  | 22413 (10792)                | ✓              |                    |              |
|  022 | [bone_density](datasets/022-bone_density.html)                                       | 2020-06-09   | 2022-12-29  | 9032 (8459)                  | ✓       